In [49]:
pip install pinecone-client==3.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 1.5 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 2.2.3
    Uninstalling pinecone-client-2.2.3:
      Successfully uninstalled pinecone-client-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [50]:
pip show pinecone-client

Name: pinecone-client
Version: 3.0.1
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /Users/pranayrajsarap/Machine Learning and Deep Learning Projects/LangChain series/End to End LLM Langchain Project using Pinecone Vector Database/venv/lib/python3.10/site-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import Libraries
import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI


/Users/pranayrajsarap/Machine Learning and Deep Learning Projects/LangChain series/End to End LLM Langchain Project using Pinecone Vector Database/venv/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [66]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os

In [5]:
## Let's Read the document
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [8]:
doc = read_doc('documents/')
len(doc)

58

In [9]:
# Divide the docs into chunks

def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc



In [33]:
documents = chunk_data(docs=doc)
len(documents)

58

In [55]:
# Embedding Technique of OPENAI
import numpy as np
model_name = "text-embedding-ada-002"
embeddings = OpenAIEmbeddings(
    model=model_name,
    api_key=os.environ['OPENAI_API_KEY']
)
#document_embeddings = [embeddings.embed_query(str(document)) for document in documents]
# doc_embeddings = [np.array(embedding) for embedding in document_embeddings]
# new_embeddings = [embedding.tolist() for embedding in doc_embeddings]



In [46]:
#len(document_embeddings)

58

In [15]:
vectors = embeddings.embed_query("how are you?")
len(vectors)

1536

## Latest Pincecone version initialization

In [67]:
# Vector Search DB in Pinecone
from pinecone import Pinecone, ServerlessSpec
pc=Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index_name = "langchainvector"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

In [64]:
pip install langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 3.0.1
    Uninstalling pinecone-client-3.0.1:
      Successfully uninstalled pinecone-client-3.0.1
Note: you may need to restart the kernel to use updated packages.


In [68]:
# Embedding Chunks to Upsert into Pinecone veector databse
import time
from langchain_pinecone import PineconeVectorStore

namespace="wondervector5000"

docsearch = PineconeVectorStore.from_documents(
    documents=documents,
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

time.sleep(1)

In [69]:
# Using Pinecone's list and query operations to look at one of the records
index = pc.Index(index_name)

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0],
        namespace=namespace,
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)

{'matches': [{'id': '056d8e7d-bf3c-47c0-83cd-5639ff1f68cc',
              'metadata': {'page': 26.0,
                           'source': 'documents/bs2023_24.pdf',
                           'text': '23 \n'
                                   ' \n'
                                   ' \n'
                                   ' \uf0b7 Permitting acquisition financing '
                                   'by IFSC Banking Units of \n'
                                   'foreign banks,  \n'
                                   '\uf0b7 Establishing a subsidiary of EXIM '
                                   'Bank for trade  \n'
                                   're-financing, \n'
                                   '\uf0b7 Amending IFSCA Act for statutory '
                                   'provisions for arbitration, \n'
                                   'ancillary services, and avoiding dual '
                                   'regulation under SEZ Act, \n'
                                 

In [70]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 10.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.5.2
    Uninstalling tiktoken-0.5.2:
      Successfully uninstalled tiktoken-0.5.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.63
    Uninstalling langsmith-0.1.63:
      Successfully uninstalled langsmith-0.1.63
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.9 requires langchain-core<0.2,>=0.1.2

In [77]:
pip install -U langchain

Note: you may need to restart the kernel to use updated packages.


In [81]:
# Buildind a Chatbot

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model="gpt-3.5-turbo",
    temperature=0.6
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = docsearch.as_retriever()
)



In [82]:
query1 = "Explain about the priorities of the budget"

In [84]:
query1_with_knowledge = qa.invoke(query1)
query1_without_knowledge = llm.invoke(query1)

print(query1_with_knowledge)
print()
print(query1_without_knowledge)

{'query': 'Explain about the priorities of the budget', 'result': "The budget outlined seven priorities, which are described as the 'Saptarishi' guiding through the Amrit Kaal. These priorities are:\n\n1) Inclusive Development\n2) Reaching the Last Mile\n3) Infrastructure and Investment\n4) Unleashing the Potential\n5) Green Growth\n6) Youth Power\n7) Financial Sector\n\nEach of these priorities aims to complement each other and contribute to the overall goal of an empowered and inclusive economy during the Amrit Kaal."}

content='The priorities of the budget refer to the specific areas or programs that a government or organization deems most important and deserving of financial resources. These priorities are typically outlined in the budget document and reflect the values and goals of the entity creating the budget.\n\nPriorities in a budget can vary depending on the specific needs and circumstances of the government or organization. Common priorities may include funding for essentia